# Sentence to vector
## Deep Neural NLP, Fall'23

Libraries

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf

Load Data

In [23]:
data = pd.read_excel("/content/train.xlsx")

FileNotFoundError: ignored

In [81]:
data[:200]

,Unnamed: 0,added,created,id,source,text,version,preprocessed_text
0,0,2018-04-03T03:50:09.862Z,1985-08-01T00:00:00.000Z,36671720,s2ag/train,Glucose-stimulated DNA Replication of the Panc...,v2,glucose stimulated dna replication of the panc...
1,1,2018-04-03T04:31:01.473Z,2010-09-28T00:00:00.000Z,39272020,s2ag/train,Secular trends of obesity in Iran between 1999...,v2,secular trends of obesity in iran between one ...
2,2,2017-08-17T21:22:18.573Z,2002-04-16T00:00:00.000Z,27500570,s2ag/train,Gating the bacterial mechanosensitive channel ...,v2,gating the bacterial mechanosensitive channel ...
3,3,2019-05-12T14:21:58.738Z,2019-04-03T00:00:00.000Z,149914560,s2ag/train,The State of Criminal Justice Educational Prog...,v2,the state of criminal justice educational prog...
4,4,2020-07-09T09:13:22.798Z,2020-06-25T00:00:00.000Z,225685510,s2ag/train,Utilizing Big Data Framework to Support Decisi...,v2,utilizing big data framework to support decisi...
...,...,...,...,...,...,...,...,...
195,195,2018-04-03T06:19:14.041Z,2007-11-01T00:00:00.000Z,24457150,s2ag/train,Glucose penetration and fluid transport throug...,v2,glucose penetration and fluid transport throug...
196,196,2021-12-23T14:11:37.767Z,2021-11-11T00:00:00.000Z,245388050,s2ag/train,Numerical Analysis of Voice Activity Detection...,v2,numerical analysis of voice activity detection...
197,197,2022-07-27T06:17:51.887Z,2022-07-26T00:00:00.000Z,251068400,s2ag/train,Simultaneous and Proportional Real-Time Myocon...,v2,simultaneous and proportional real time myocon...
198,198,2018-04-03T03:29:52.322Z,1992-01-01T00:00:00.000Z,5788950,s2ag/train,Second transplants for patients with chronic m...,v2,second transplants for patients with chronic m...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         3000 non-null   int64 
 1   added              3000 non-null   object
 2   created            3000 non-null   object
 3   id                 3000 non-null   int64 
 4   source             3000 non-null   object
 5   text               3000 non-null   object
 6   version            3000 non-null   object
 7   preprocessed_text  3000 non-null   object
dtypes: int64(2), object(6)
memory usage: 187.6+ KB


Get all the sentences

In [7]:
data.preprocessed_text = data.preprocessed_text.apply(lambda x : x.replace('[','').replace(']','').replace("'",""))

In [8]:
def get_train(docs, window_size):
    sentences = []

    for doc in docs:
        for i in range(window_size): sentences.append("<doc>")
        sentences.extend(doc.split(', '))

    for i in range(window_size): sentences.append("<doc>")

    with open('sentences.txt','w') as tfile:
        tfile.write('\n'.join(sentences))

    vocab = set()

    for sentence in sentences:
        vocab.update(sentence.split(' '))

    vocab = list(vocab)
    with open('vocab.txt','w') as tfile:
        tfile.write('\n'.join(vocab))

    target = pd.DataFrame(np.zeros((len(sentences), len(vocab))), dtype=np.int16)
    target.columns = vocab

    for s in range(len(sentences)):
        for word in sentences[s].split(' '):
            target.at[s, word] = 1

    target['<doc>'] = [0]*len(sentences)

    target.to_csv('target.csv')

    train = pd.DataFrame(columns=target.columns)
    y_label = pd.DataFrame(columns=target.columns)

    print("creating features and lables")

    for i in target.index:
        if sentences[i] != '<doc>':
            for j in range(max(0, i - window_size), min(i + window_size + 1, len(target))):
                if i != j and sentences[j] != '<doc>':
                    train.loc[len(train)] = target.loc[i]
                    y_label.loc[len(y_label)] = target.loc[j]

    return train, y_label

In [9]:
train, y_label = get_train(data.preprocessed_text[:200], 2)

creating features and lables


In [10]:
train.to_csv('train.csv')
y_label.to_csv('ylabel.csv')

Training Sent2Vec

In [6]:
train = pd.read_csv("train.csv", index_col=0)
y_label = pd.read_csv("ylabel.csv", index_col=0)

In [8]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [10]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss',  # Metric to monitor
                               mode='min',           # Minimize the metric
                               patience=5,           # Number of epochs with no improvement
                               verbose=1)            # Print messages when stopping


In [11]:
batch_size = 8
train_dataset = tf.data.Dataset.from_tensor_slices((train, y_label))
train_dataset = train_dataset.batch(batch_size)

In [12]:
vocab_size = len(train.columns)

In [13]:
lines = None
with open('sentences.txt') as f:
    lines = f.readlines()

In [14]:
target = pd.read_csv('target.csv', index_col=0)

In [15]:
# Hyperparameters
embedding_dim = 50
learning_rate = 0.01
epochs = 100

# Define the Skip-Gram model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    tf.keras.layers.Dense(units=vocab_size, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy')

model.fit(train_dataset, epochs=epochs, callbacks=[early_stopping])

# Get the learned word embeddings
weights = model.layers[0].get_weights()[0]

Epoch 1/100
253/253 [==============================] - 50s 175ms/step - loss: nan
Epoch 2/100
253/253 [==============================] - 44s 173ms/step - loss: nan
Epoch 3/100
253/253 [==============================] - 44s 174ms/step - loss: nan
Epoch 4/100
217/253 [========================>.....] - ETA: 6s - loss: nan

KeyboardInterrupt: ignored

In [ ]:
embeds = target.dot(weights)

In [ ]:
embeds.to_csv('embeds.csv')

In [2]:
vocab = train.columns

NameError: ignored

In [3]:
def embed(sent, vocab, w):
    sent_embed = pd.Series(0, index=vocab, dtype=int)
    for word in sent.split(' '):
        sent_embed[word] = 1
    return sent_embed.dot(w)

In [4]:
def similar(sent, vocab, w, embeddings, k, sentences):
    sent_emb = embed(sent, vocab, w)
    from sklearn.metrics.pairwise import cosine_similarity

    # Sample dataset of vectors (each row represents a vector)
    dataset = embeddings

    # Target vector for which you want to find the nearest neighbors
    target_vector = sent_emb

    # Calculate cosine similarity between the target vector and all vectors in the dataset
    similarities = cosine_similarity([target_vector], dataset)

    # Flatten the similarities array and get the indices of the top-k values
    nearest_indices = np.argsort(-similarities, axis=None)[:k]

    # Retrieve the nearest neighbors based on the indices
    for ind in nearest_indices:
        print(sentences[ind])


In [ ]:
similar("in the correlation analysis", vocab, weights, embeds, 5, lines)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

In [ ]:
pca_embeds = pca.fit_transform(embeds)

In [ ]:
pca_x = list(pair[0] for pair in pca_embeds[:50])
pca_y = list(pair[1] for pair in pca_embeds[:50])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(pca_x, pca_y)